# hw4. Оценка качества исправления ошибок


### 1. Trimmomatic (просто режет) 
Запуск:

``$ TrimmomaticPE -phred33 data/4/ecoli_400K_err_1.fastq data/4/ecoli_400K_err_2.fastq -baseout /home/toharhymes/work/ngs_ib_spring2020/project_four_errors/data/4/trimm_ecoli_400K LEADING:20 TRAILING:20  SLIDINGWINDOW:10:20 MINLEN:20``

Далее будем использовать только `trimm_ecoli_400K_1P` и `trimm_ecoli_400K_2P` (так как они у обоих ок).

Проиндексируем референс:

``$ bwa index data/4/MG1655-K12.first400K.fasta``

Выровняем скорректированные риды, и риды с ошибками:<br>
``$ bwa mem data/4/MG1655-K12.first400K.fasta data/4/trimm_ecoli_400K_1P data/4/trimm_ecoli_400K_2P > data/4/alignment_correct.sam`` <br>
``$ samtools flagstat data/4/alignment_correct.sam 
2689072 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2687824 + 0 mapped (99.95% : N/A)
2689072 + 0 paired in sequencing
1344536 + 0 read1
1344536 + 0 read2
2685506 + 0 properly paired (99.87% : N/A)
2686782 + 0 with itself and mate mapped
1042 + 0 singletons (0.04% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)``



``$ bwa mem data/4/MG1655-K12.first400K.fasta data/4/ecoli_400K_err_1.fastq data/4/ecoli_400K_err_2.fastq > data/4/alignment_error.sam ``<br>
``$ samtools flagstat data/4/alignment_error.sam 
2763204 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2762617 + 0 mapped (99.98% : N/A)
2763204 + 0 paired in sequencing
1381602 + 0 read1
1381602 + 0 read2
2760820 + 0 properly paired (99.91% : N/A)
2762132 + 0 with itself and mate mapped
485 + 0 singletons (0.02% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)
``

### 2. Spades (BayesHammer)

``$ spades --only-error-correction --pe1-1 data/4/ecoli_400K_err_1.fastq.gz --pe1-2 data/4/ecoli_400K_err_2.fastq.gz -o data/4/corrected_spades``

``$ bwa mem data/4/MG1655-K12.first400K.fasta data/4/corrected_spades/corrected/ecoli_400K_err_1.fastq.00.0_0.cor.fastq.gz data/4/corrected_spades/corrected/ecoli_400K_err_2.fastq.00.0_0.cor.fastq.gz > data/4/corrected_alignment_spades.sam
``

``$ samtools flagstat data/4/corrected_alignment_spades.sam ``

``
2716054 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
2715944 + 0 mapped (100.00% : N/A)
2716054 + 0 paired in sequencing
1358027 + 0 read1
1358027 + 0 read2
2714586 + 0 properly paired (99.95% : N/A)
2715876 + 0 with itself and mate mapped
68 + 0 singletons (0.00% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=
``

In [3]:
import pysam
import numpy as np
import pandas as pd

In [23]:
# возвращает словарь: индекс в референсе - значение(буква) в последовательности для рида
def reference_read_mapping(read):
    mapping_dict = {}
#     print(read.get_aligned_pairs(matches_only=False))
    for read_i, ref_i in read.get_aligned_pairs():
        mapping_dict.update({ref_i: None if read_i is None else read.query_sequence[read_i]})
    return mapping_dict

# возвращает словарь: индекс в референсе - значение(буква) в референсе
def reference_mapping(read):
    mapping_dict = {}
    for read_i, ref_i, ref_l in read.get_aligned_pairs(with_seq = True):
        mapping_dict.update({ref_i: ref_l})
    return mapping_dict


def count(correct_sam_file, raw_sam_file):
    cor_sam = pysam.AlignmentFile(correct_sam_file, "r")
    raw_sam = pysam.AlignmentFile(raw_sam_file, "r")
    flag = True
    answer = np.zeros((2,3), dtype=int)
    try:
        while flag:
            cor_read = next(cor_sam)
            raw_read = next(raw_sam)
            if not cor_read.is_unmapped and not raw_read.is_unmapped and not  cor_read.is_supplementary and not raw_read.is_supplementary:
                while cor_read.query_name != raw_read.query_name:
#                     for pair in raw_read.get_aligned_pairs():
#                         if pair[0] != None and pair[1] != None:
#                             # Incorrectly removed base - убрал, а не должен был - сырой смапился 
#                             answer[1][2] += 1
#                         else:
#                             # Correctly remove base - убрал правильно, так как не мапился, или мапился неверно
#                             answer[0][2] += 1   
                    raw_read = next(raw_sam)
    
                # индекс в референсе : буква (или None) в скорректированном риде
                cor_mapping = reference_read_mapping(cor_read)
                # индекс в референсе : буква (или None) в нескорректированном риде
                raw_mapping = reference_read_mapping(raw_read)
                # индекс в референсе : буква (или None) в референсе
                ref_mapping = reference_mapping(cor_read)
                
                for index in cor_mapping:
                    cor = cor_mapping[index]
                    raw = raw_mapping[index] if index in raw_mapping else None
                    ref = ref_mapping[index] if index in ref_mapping else None
                    # if base is absent in corrected=> the third column(index = 2):
                    if cor == 'N':
                        answer[int(raw == ref)][2] += 1
                        continue
                    answer[int(raw==ref)][int(cor == ref)] += 1
        
    except StopIteration:
        return pd.DataFrame(answer, columns = ['Error in corrected reads','Correct base in corrected reads', 'Base is absent in corrected reads'], index = ['Error in raw data', 'Correct base in raw data'])
    

### Trimmomatic

In [26]:
count('data/4/alignment_correct.sam', 'data/4/alignment_error.sam')

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,791466,1962606,137091
Correct base in raw data,5353,241117011,8


In [27]:
count('data/4/corrected_alignment_spades.sam', 'data/4/alignment_error.sam')

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,235309,2894143,726431
Correct base in raw data,8306,246936100,617366


#### Как делал:

По индексу в референсе составил три словаря: индекс - буква в референсе, индекс - буква в неисправленном риде, индекс - буква в исправленном риде, дальше по ним итерировался и искал такие ошибки:

![](table.png)

0я строка - неисправленный != референсу, а 1я строка неисправленный равен референсу, поэтому строка это условие `ref==raw`. Со столбцами аналогично. в 0м скорректированный не равен референсу (у нас ошибка), а в первом равен, поэтому условие: `cor -- ref`. Ну а если основания нету в исправленном (== 'N'), то это третий столбец